In [1]:
import cv2 as cv

In [3]:
import numpy as np

In [4]:
import dlib

In [7]:
from imutils import face_utils

In [9]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [10]:
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0,0,0)

In [13]:
def dist(a, b):
    dst = np.linalg.norm(a-b)
    return dst

In [15]:
def blinked(a,b,c,d,e,f):
    up = dist(b, d) + dist(c,e)
    down = dist(a, f)
    ratio = up/(2.0*down)

    if ratio>0.3:
        return 2
    elif 0.3 >= ratio > 0.15:
        return 1
    else:
        return 0

In [ ]:
cap = cv.VideoCapture(0)
while True:
    _, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        face_frame = frame.copy()
        cv.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)
        left_blink = blinked(landmarks[36],landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42],landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        
        if left_blink==0 and right_blink == 0:
            sleep+=1
            drowsy = 0
            active = 0
            if sleep>6:
                status = "SLEEPING!"
                color = (255, 0, 0)
        
        elif left_blink==1 and right_blink==1:
            sleep = 0
            active = 0
            drowsy+=1
            if drowsy>6:
                status = "DROWSY!"
                color = (0,0,255)
        
        else:
            drowsy = 0
            sleep = 0
            active+=1
            if active>6:
                status = "ACTIVE!"
                color = (0, 255, 0)
        
        cv.putText(frame, status, (100,100), cv.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        for n in range(0, 68):
            (x,y) = landmarks[n]
            cv.circle(face_frame, (x, y), 1, (255, 255, 255), -1)
        
        cv.imshow("Frame", frame)
        cv.imshow("Result of detector", face_frame)
        key = cv.waitKey(1)
        if key == 27:
            break

2025-02-18 16:52:44.148 python[96824:9701886] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-18 16:52:44.148 python[96824:9701886] +[IMKInputSession subclass]: chose IMKInputSession_Modern
